In [1]:
#The following file paths are all absolute paths. You can replace them with relative paths at runtime, and the files are located in their respective folders.
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import gym
import matplotlib.pyplot as plt
import random
import argparse
from collections import OrderedDict
from copy import copy
from tqdm import tqdm, trange
import scipy
import scipy.linalg
from scipy.integrate import odeint
import sys
import os
import csv 
sys.path.append("/media/serena/study/Vscode_works/python_Vscode/mamba/Nonlinear_LSPN/Predict_Model_Train/")
sys.path.append("/media/serena/study/Vscode_works/python_Vscode/mamba/Nonlinear_LSPN/utility_LSPN/")
from LSPN_test import LSPN_Mamba
os.environ['KMP_DUPLICATE_LIB_OK'] = "TRUE"


/home/serena/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
Methods = ["KoopmanDerivative","KoopmanRBF",\
            "KNonlinear","KNonlinearRNN","KoopmanU",\
            "KoopmanNonlinearA","KoopmanNonlinear",\
            "KNonlinearmamba"]
Method_names = ["KoopmanDerivative","KoopmanRBF",\
            "KDNN","KRNN","DKUC(ours)",\
            "DKAC(ours)","DKN(ours)",\
            "KNonlinearmamba"]

In [ ]:
def read_henon_map_dataset_original_shape(file_path):
    data = np.load(file_path)
    return data

def eval_err(suffix,method_index):
    # method_index = 0
    method = Methods[method_index]
    root_path = "/media/serena/study/Vscode_works/python_Vscode/mamba/Nonlinear_LSPN/DATA/Mamba_data_raw/"+suffix
    print(method)
    if method.endswith("KNonlinear"):
        import Learn_Knonlinear as lka
    elif method.endswith("KNonlinearRNN"):
        import Learn_Knonlinear_RNN as lka
    elif method.endswith("KoopmanNonlinear"):
        import Learn_KoopmanNonlinear_with_KlinearEig as lka
    elif method.endswith("KoopmanNonlinearA"):
        import Learn_KoopmanNonlinearA_with_KlinearEig as lka
    elif method.endswith("KoopmanU"):
        import Learn_Koopman_with_KlinearEig as lka
    elif method.endswith("KNonlinearmamba"):
        import Learn_Knonlinear_mamba_henon_map as lka   
    for file in os.listdir(root_path):
        if file.startswith(method+"_"+"henon_map") and file.endswith(".pth"):
            model_path = file  
    dicts = torch.load(root_path+"/"+model_path,map_location=torch.device('cpu'))
    state_dict = dicts["model"]
    if method.endswith("KNonlinearRNN"):
        net = lka.Network(input_size=3,output_size=3,hidden_dim=128, n_layers=3)
    elif method.endswith("KNonlinearmamba"):
        net = LSPN_Mamba(
        d_model=2, 
        d_state=8, 
        d_conv=4, 
        expand=4,  
    ).to("cuda")
    net.load_state_dict(state_dict)
    total_params = sum(p.numel() for p in net.parameters())
    print(f"Total parameters: {total_params}")
    # net.cuda()
    net.double()
    Samples = 500
    steps = 200
    random.seed(2022)
    np.random.seed(2022)
    times = 4 
    max_loss_all = np.zeros((times,steps-1))
    mean_loss_all = np.zeros((times,steps-1))
    with torch.no_grad():
        for i in trange(times, desc="predicting_van_der_pol", unit="times"):
            X_original_shape = read_henon_map_dataset_original_shape('/media/serena/study/Vscode_works/python_Vscode/mamba/Nonlinear_LSPN/utility_LSPN/henon_map_data_filtered.npy')
            test_data = X_original_shape[-Samples:, :steps, :]
            np.save("/media/serena/study/Vscode_works/python_Vscode/mamba/Nonlinear_LSPN/DATA/LSPN_predict_data/henon_map"+suffix+".npy",test_data)
            max_loss,mean_loss = lka.K_loss(test_data,net)
            max_loss_all[i] = max_loss.reshape(-1)
            mean_loss_all[i] = mean_loss.reshape(-1)
    max_mean = np.mean(max_loss_all,axis=0)
    max_std = np.std(max_loss_all,axis=0)
    mean_mean =  np.mean(mean_loss_all,axis=0)
    mean_std =  np.std(mean_loss_all,axis=0)
    np.save("/media/serena/study/Vscode_works/python_Vscode/mamba/Nonlinear_LSPN/DATA/LSPN_rossler_data/henon_map"+suffix+"_"+method+".npy",np.array([max_mean,max_std,mean_mean,mean_std]))
    return max_mean,max_std,mean_mean,mean_std

In [ ]:
suffix = "mamba_testhenon_map1"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

for i in [7]:
    #i = 2
    eval_err(suffix,method_index=i)


In [ ]:
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
font = {'size'   : 12}

plt.rc('font', **font)
# markers = ['*','+','*','+','*','+','*']
title = "van_der_pol"
# compare = "max"
compare = "mean"
for i in [7]:
    file_path = "/media/serena/study/Vscode_works/python_Vscode/mamba/Nonlinear_LSPN/DATA/LSPN_rossler_data/henon_map"+suffix+"_"+Methods[i]+".npy"
    data = np.load(file_path)
    plt.plot(np.log10(data[2]),'*-',color = colors[i],label=Methods[i],linewidth=1.5)
plt.legend()
# plt.grid(linestyle='-.')
plt.xlabel("Steps",fontsize=12)
plt.ylabel("Log Err",fontsize=12)
plt.title(title,fontsize=15)
# plt.savefig("D:/毕业设计/论文/pictures/sizeNN/"+env_name+"_TEST_"+compare+"_new1.png",dpi=400)

In [5]:
def read_henon_map_dataset_original_shape(file_path):
    data = np.load(file_path)
    return data

def draw_LSPN_only_eval_err(suffix,method_index,env_name = "henon_map"):
    # method_index = 0
    method = Methods[method_index]
    root_path = "/media/serena/study/Vscode_works/python_Vscode/mamba/Nonlinear_LSPN/DATA/Mamba_data_raw/"+suffix
    print(method)
    if method.endswith("KNonlinear"):
        import Learn_Knonlinear as lka
    elif method.endswith("KNonlinearRNN"):
        import Learn_Knonlinear_RNN as lka
    elif method.endswith("KoopmanNonlinear"):
        import Learn_KoopmanNonlinear_with_KlinearEig as lka
    elif method.endswith("KoopmanNonlinearA"):
        import Learn_KoopmanNonlinearA_with_KlinearEig as lka
    elif method.endswith("KoopmanU"):
        import Learn_Koopman_with_KlinearEig as lka
    elif method.endswith("KNonlinearmamba"):
        import Learn_Knonlinear_mamba_henon_map as lka   
    for file in os.listdir(root_path):
        if file.startswith(method+"_"+"henon_map") and file.endswith(".pth"):
            model_path = file  
    dicts = torch.load(root_path+"/"+model_path,map_location=torch.device('cpu'))
    state_dict = dicts["model"]
    if method.endswith("KNonlinearRNN"):
        net = lka.Network(input_size=3,output_size=3,hidden_dim=128, n_layers=3)
    elif method.endswith("KNonlinearmamba"):
        net = LSPN_Mamba(
        d_model=2, 
        d_state=8, 
        d_conv=4, 
        expand=4,  
    ).to("cuda")
    net.load_state_dict(state_dict)
    total_params = sum(p.numel() for p in net.parameters())
    print(f"Total parameters: {total_params}")
    # net.cuda()
    net.double()
    Samples = 50
    steps = 51
    random.seed(2022)
    np.random.seed(2022)
    times = 1
    max_loss_all = np.zeros((Samples,steps-1))
    mean_loss_all = np.zeros((Samples,steps-1))
    with torch.no_grad():
        X_original_shape = read_henon_map_dataset_original_shape('/media/serena/study/Vscode_works/python_Vscode/mamba/Nonlinear_LSPN/utility_LSPN/henon_map_data_filtered.npy')
        for i in trange(Samples, desc="predicting_henon_map", unit="times"):
            test_data = X_original_shape[i:i+1, :steps, :]
            np.save("/media/serena/study/Vscode_works/python_Vscode/mamba/Nonlinear_LSPN/DATA/LSPN_predict_data/middle_data/draw_LSPN_only{}{}{}.npy".format(i,suffix,env_name),test_data)
            max_loss,mean_loss = lka.K_loss(test_data,net)
            max_loss,mean_loss = np.log10(max_loss), np.log10(mean_loss)
            max_loss_all[i] = max_loss.reshape(-1)
            mean_loss_all[i] = mean_loss.reshape(-1)
    print(max_loss_all.shape)
    print(mean_loss_all.shape)
    np.save("/media/serena/study/Vscode_works/python_Vscode/mamba/Nonlinear_LSPN/DATA/LSPN_predict_data/draw_LSPN_only{}".format(Samples)+suffix+env_name+"_"+method+".npy",np.array(mean_loss_all))
    return mean_loss_all

In [6]:
suffix = "mamba_testhenon_map1"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   
env_name = "henon_map"
for i in [7]:
    #i = 2
    draw_LSPN_only_eval_err(suffix,method_index=i,env_name = "henon_map")


KNonlinearmamba
Total parameters: 312


predicting_henon_map: 100%|██████████| 50/50 [00:00<00:00, 277.02times/s]

(50, 50)
(50, 50)
